Example URLs:

bielefeld1
http://www.dfld.de/Mess/StatMon.php?R=012&S=002&D=04.02.2022&AS=537990914&AiO=RG89RCRATD1HJEBSPTEyJEBTPTAwMiRASz0wJEBEPTA0LjAyLjIwMjIkQFNUPTQkQFo9MDA6MDA6MDAkQFpUPSRAQT0xJEBQcz0kQE1Ocj0wJEBOb09wdD0w

bielefeld2
http://www.dfld.de/Mess/StatMon.php?R=012&S=004&D=04.02.2022&AS=537990914&AiO=RG89RCRATD1HJEBSPTEyJEBTPTAwNCRASz0wJEBEPTA0LjAyLjIwMjIkQFNUPTQkQFo9MDA6MDA6MDAkQFpUPSRAQT0xJEBQcz0kQE1Ocj0wJEBOb09wdD0w


hh - niendorf
http://www.dfld.de/Mess/StatMon.php?R=016&S=007&D=04.02.2022&AS=537990914&AiO=RG89RCRATD1HJEBSPTE2JEBTPTAwNyRASz0wJEBEPTA0LjAyLjIwMjIkQFNUPTQkQFo9MDA6MDA6MDAkQFpUPSRAQT0xJEBQcz0kQE1Ocj0wJEBOb09wdD0w
http://www.dfld.de/Mess/StatMon.php?R=016&S=007&D=04.01.2022&AS=269555458


laerm_cities = {40: Aachen, 2:Berlin, 12:Bielefeld, 31:Braunschweig, 24:Bremen, 39:Dinslaken, 4:Duesseldorf, 14:Dortmund, 27:Dresden, 1:Frankfurt, 16:Hamburg, 5:Hannover, 3:Koeln/Bonn, 9:Leipzig, 18:Muenchen, 28:Nuernberg, 6:Ramstein, 15:Saarbruecken}

### Download Noise-Level-Values per Day http://www.dfld.de/ 

In [93]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Load DFLD Station IDs
dfld_stations = pd.read_csv('data/DFLD/DFLD_Region_Station.csv')
stations_length = dfld_stations.shape[0]

#stations_length = 12

df_city = pd.DataFrame([])

monate = ['01','02','03','04','05','06','07','08','09','10','11','12']
jahre = ['2017','2018','2019','2020','2021']
#monate = ['01','02','03']
#jahre = ['2021']


for year_url in jahre: 
    for month_url in monate:
        date_url = f"{month_url}.{year_url}"
        print(date_url)

        for num in range(stations_length):
            stations_nummer = dfld_stations.loc[num, 'Stations_nummer']
            regions_nummer = dfld_stations.loc[num, 'Regions_nummer']
            
            #stations_nummer = 282
            #regions_nummer = 3


            # URL Definition
            url = 'http://www.dfld.de/Mess/StatMon.php?R=%s&S=%s&D=04.%s&AS=537990914' %(regions_nummer, stations_nummer, date_url)
            print(url)
            res = requests.get(url)
            x=res.text

            # Turn requested HTML-file into dataframe
            soup = BeautifulSoup(x, "html.parser")
            table = soup.find_all('table')
            df = pd.read_html(str(table),thousands='.')[0]
            
            # Check if no data available continue with next element 
            if pd.isna(df.loc[1, 1]) == True and pd.isna(df.loc[1, 2]) == True :
                continue

            # Setting the header corret 
            df.columns = df.iloc[4]
            df_body = df

            print("regions_nummer", regions_nummer, "stations_nummer", stations_nummer)
            #exceptions for r 1,3,9 due to different formats

            if  regions_nummer == 1:
            #if  regions_nummer in r_except:
                df_body.drop('LKernnacht(00‑05+23-24)', inplace=True, axis=1)
                df_body['LNacht(05‑06+22‑23)'] = df_body['LNacht(00‑06+22-24)'].values
                df_body.drop('LNacht(00‑06+22-24)', inplace=True, axis=1)
                df_body.rename(columns={'LNacht(05‑06+22‑23)':'LNacht(00‑06+22-24)'},inplace=True)
                df_body.rename(columns={'NAT (68 dBA)(Nacht+Kernnacht)':'NAT (68 dBA)(Nacht)'},inplace=True)
                df_body = df_body.iloc[5: , :8]
            elif regions_nummer == 3:
                df_body.drop('LKernnacht(00‑05)', inplace=True, axis=1)
                df_body['LNacht(05‑06+22-24)'] = df_body['LNacht(00‑06+22-24)'].values
                df_body.drop('LNacht(00‑06+22-24)', inplace=True, axis=1)
                df_body.rename(columns={'LNacht(05‑06+22-24)':'LNacht(00‑06+22-24)'},inplace=True)
                df_body.rename(columns={'NAT (72 dBA)(Nacht+Kernnacht)':'NAT (68 dBA)(Nacht)'},inplace=True)
                df_body = df_body.iloc[5: , :8]
            elif regions_nummer == 9:
                df_body.drop('LKernnacht(00‑05)', inplace=True, axis=1)
                df_body['LNacht(05‑06+22-24)'] = df_body['LNacht(00‑06+22-24)'].values
                df_body.drop('LNacht(00‑06+22-24)', inplace=True, axis=1)
                df_body.rename(columns={'LNacht(05‑06+22-24)':'LNacht(00‑06+22-24)'},inplace=True)
                df_body.rename(columns={'NAT (68 dBA)(Nacht+Kernnacht)':'NAT (68 dBA)(Nacht)'},inplace=True)
                df_body = df_body.iloc[5: , :8]
            else:
                # selecting the needed columns
                df_body = df_body.iloc[5: , :8]
                
            df_body['r_number'] = regions_nummer
            df_body['region'] = dfld_stations.loc[num, 'Region']
            df_body['s_number'] = stations_nummer
            df_body['station'] = dfld_stations.loc[num, 'Station']
            df_city = pd.concat([df_city, df_body])
df_city





01.2017
http://www.dfld.de/Mess/StatMon.php?R=40&S=1&D=04.01.2017&AS=537990914
http://www.dfld.de/Mess/StatMon.php?R=2&S=101&D=04.01.2017&AS=537990914
http://www.dfld.de/Mess/StatMon.php?R=2&S=62&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 62
http://www.dfld.de/Mess/StatMon.php?R=2&S=48&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 48
http://www.dfld.de/Mess/StatMon.php?R=2&S=69&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 69
http://www.dfld.de/Mess/StatMon.php?R=2&S=32&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 32
http://www.dfld.de/Mess/StatMon.php?R=2&S=109&D=04.01.2017&AS=537990914
http://www.dfld.de/Mess/StatMon.php?R=2&S=87&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 87
http://www.dfld.de/Mess/StatMon.php?R=2&S=20&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 20
http://www.dfld.de/Mess/StatMon.php?R=2&S=14&D=04.01.2017&AS=537990914
regions_nummer 2 stations_nummer 14
http://www.dfld.de/Mess/Stat

4,Datum,LTag(06‑18),LTagrand(18‑22),LNacht(00‑06+22-24),Lden(00-24),LTag(06‑22),Ldn(00-24),NAT (68 dBA)(Nacht),r_number,region,s_number,station
5,01012017,----,"41,3","34,9","42,4","35,4","40,9",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
6,02012017,----,"36,4","34,9","40,7","30,3","40,0",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
7,03012017,----,"33,1",----,"36,0",----,"35,1",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
8,04012017,"36,9","36,5","34,5","41,5","36,8","41,0",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
9,05012017,----,"35,5",----,"33,0",----,----,0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
...,...,...,...,...,...,...,...,...,...,...,...,...
31,27122021,"38,6",----,----,"35,6","37,4","35,6",0,15,Saarbruecken,1,Saarbruecken
32,28122021,"31,6",----,----,----,"30,3",----,0,15,Saarbruecken,1,Saarbruecken
33,29122021,----,"39,8",----,"37,4","34,8","33,0",0,15,Saarbruecken,1,Saarbruecken
34,30122021,----,----,----,----,----,----,0,15,Saarbruecken,1,Saarbruecken


### Cleaning Noise per day


In [99]:
import datetime
df_city.rename(columns={'LTag(06‑18)':'LTag(18)', 'LTagrand(18‑22)':'LTagrand','LNacht(00‑06+22-24)':'LNacht','Lden(00-24)':'Lden','LTag(06‑22)':'LTag(22)','Ldn(00-24)':'Ldn','NAT (68 dBA)(Nacht)':'NAT'},inplace=True)
df_city['Datum'] = pd.to_datetime(df_city['Datum'],format='%d%m%Y')

# transform Income column to a numerical
df_city['LTag(18)'] = df_city['LTag(18)'].str.replace('----', '')
df_city['LTag(18)'] = df_city['LTag(18)'].str.replace(',', '.')
df_city['LTag(18)'] = pd.to_numeric(df_city['LTag(18)'])

df_city['LTagrand'] = df_city['LTagrand'].str.replace('----', '')
df_city['LTagrand'] = df_city['LTagrand'].str.replace(',', '.')
df_city['LTagrand'] = pd.to_numeric(df_city['LTagrand'])

df_city['LNacht'] = df_city['LNacht'].str.replace('----', '')
df_city['LNacht'] = df_city['LNacht'].str.replace(',', '.')
df_city['LNacht'] = pd.to_numeric(df_city['LNacht'])

df_city['Lden'] = df_city['Lden'].str.replace('----', '')
df_city['Lden'] = df_city['Lden'].str.replace(',', '.')
df_city['Lden'] = pd.to_numeric(df_city['Lden'])

df_city['LTag(22)'] = df_city['LTag(22)'].str.replace('----', '')
df_city['LTag(22)'] = df_city['LTag(22)'].str.replace(',', '.')
df_city['LTag(22)'] = pd.to_numeric(df_city['LTag(22)'])

df_city['Ldn'] = df_city['Ldn'].str.replace('----', '')
df_city['Ldn'] = df_city['Ldn'].str.replace(',', '.')
df_city['Ldn'] = pd.to_numeric(df_city['Ldn'])

df_city['NAT'] = df_city['NAT'].str.replace('----', '')
df_city['NAT'] = df_city['NAT'].str.replace(',', '.')
df_city['NAT'] = pd.to_numeric(df_city['NAT'])

df_city.info()
df_city.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405796 entries, 5 to 35
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datum     405796 non-null  datetime64[ns]
 1   LTag(18)  252941 non-null  float64       
 2   LTagrand  235324 non-null  float64       
 3   LNacht    176194 non-null  float64       
 4   Lden      291211 non-null  float64       
 5   LTag(22)  267268 non-null  float64       
 6   Ldn       283670 non-null  float64       
 7   NAT       395048 non-null  float64       
 8   r_number  405796 non-null  int64         
 9   region    405796 non-null  object        
 10  s_number  405796 non-null  int64         
 11  station   405796 non-null  object        
dtypes: datetime64[ns](1), float64(7), int64(2), object(2)
memory usage: 40.2+ MB


4,Datum,LTag(18),LTagrand,LNacht,Lden,LTag(22),Ldn,NAT,r_number,region,s_number,station
5,2017-01-01,NaN,41.3,34.9,42.4,35.4,40.9,0.0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
6,2017-01-02,NaN,36.4,34.9,40.7,30.3,40.0,0.0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
7,2017-01-03,NaN,33.1,NaN,36.0,NaN,35.1,0.0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
8,2017-01-04,36.9,36.5,34.5,41.5,36.8,41.0,0.0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx
9,2017-01-05,NaN,35.5,NaN,33.0,NaN,NaN,0.0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx


### pro jahr (durchschnitt pro monat)

In [108]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Load DFLD Station IDs
dfld_stations = pd.read_csv('data/DFLD/DFLD_Region_Station.csv')
stations_length = dfld_stations.shape[0]

df_city_year = pd.DataFrame([])

jahre = ['2017','2018','2019','2020','2021']
monate = ['12']


for year_url in jahre: 
    for month_url in monate:
        date_url = f"{month_url}.{year_url}"
        print(date_url)

        for num in range(stations_length):
            stations_nummer = dfld_stations.loc[num, 'Stations_nummer']
            regions_nummer = dfld_stations.loc[num, 'Regions_nummer']
            

            # URL Definition
            url ='http://www.dfld.de/Mess/StatJah.php?R=%s&S=%s&D=04.%s&AS=537992964' %(regions_nummer, stations_nummer, date_url)
            print(url)
            res = requests.get(url)
            x=res.text

            # Turn requested HTML-file into dataframe
            soup = BeautifulSoup(x, "html.parser")
            table = soup.find_all('table')
            df = pd.read_html(str(table),thousands='.')[0]
            
            # Check if no data available continue with next element 
            if pd.isna(df.loc[1, 1]) == True and pd.isna(df.loc[1, 2]) == True :
                continue

            # Setting the header corret 
            df.columns = df.iloc[4]
            df_body = df

            #exceptions for r 1,3,9 due to different formats

            if  regions_nummer == 1:
                df_body.drop('LKernnacht(00‑05+23-24)', inplace=True, axis=1)
                df_body['LNacht(05‑06+22‑23)'] = df_body['LNacht(00‑06+22-24)'].values
                df_body.drop('LNacht(00‑06+22-24)', inplace=True, axis=1)
                df_body.rename(columns={'LNacht(05‑06+22‑23)':'LNacht(00‑06+22-24)'},inplace=True)
                df_body.rename(columns={'NAT (68 dBA)(Nacht+Kernnacht)':'NAT (68 dBA)(Nacht)'},inplace=True)
                df_body = df_body.iloc[5: , :8]
            elif regions_nummer == 3:
                df_body.drop('LKernnacht(00‑05)', inplace=True, axis=1)
                df_body['LNacht(05‑06+22-24)'] = df_body['LNacht(00‑06+22-24)'].values
                df_body.drop('LNacht(00‑06+22-24)', inplace=True, axis=1)
                df_body.rename(columns={'LNacht(05‑06+22-24)':'LNacht(00‑06+22-24)'},inplace=True)
                df_body.rename(columns={'NAT (72 dBA)(Nacht+Kernnacht)':'NAT (68 dBA)(Nacht)'},inplace=True)
                df_body = df_body.iloc[5: , :8]
            elif regions_nummer == 9:
                df_body.drop('LKernnacht(00‑05)', inplace=True, axis=1)
                df_body['LNacht(05‑06+22-24)'] = df_body['LNacht(00‑06+22-24)'].values
                df_body.drop('LNacht(00‑06+22-24)', inplace=True, axis=1)
                df_body.rename(columns={'LNacht(05‑06+22-24)':'LNacht(00‑06+22-24)'},inplace=True)
                df_body.rename(columns={'NAT (68 dBA)(Nacht+Kernnacht)':'NAT (68 dBA)(Nacht)'},inplace=True)
                df_body = df_body.iloc[5: , :8]
            else:
                # selecting the needed columns
                df_body = df_body.iloc[5: , :8]
                
            df_body['r_number'] = regions_nummer
            df_body['region'] = dfld_stations.loc[num, 'Region']
            df_body['s_number'] = stations_nummer
            df_body['station'] = dfld_stations.loc[num, 'Station']
            df_body['year'] = year_url
            
            df_city_year = pd.concat([df_city_year, df_body])
df_city_year


12.2017
http://www.dfld.de/Mess/StatJah.php?R=40&S=1&D=04.12.2017&AS=537992964
http://www.dfld.de/Mess/StatJah.php?R=2&S=101&D=04.12.2017&AS=537992964
http://www.dfld.de/Mess/StatJah.php?R=2&S=62&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 62
http://www.dfld.de/Mess/StatJah.php?R=2&S=48&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 48
http://www.dfld.de/Mess/StatJah.php?R=2&S=69&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 69
http://www.dfld.de/Mess/StatJah.php?R=2&S=32&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 32
http://www.dfld.de/Mess/StatJah.php?R=2&S=109&D=04.12.2017&AS=537992964
http://www.dfld.de/Mess/StatJah.php?R=2&S=87&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 87
http://www.dfld.de/Mess/StatJah.php?R=2&S=20&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 20
http://www.dfld.de/Mess/StatJah.php?R=2&S=14&D=04.12.2017&AS=537992964
regions_nummer 2 stations_nummer 14
http://www.dfld.de/Mess/Stat

4,Monat,LTag(06‑18),LTagrand(18‑22),LNacht(00‑06+22-24),Lden(00-24),LTag(06‑22),Ldn(00-24),NAT (68 dBA)(Nacht),r_number,region,s_number,station,year
5,Januar,----,"35,2","30,5","37,7","32,0","36,8",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017
6,Februar,----,"34,3",----,"35,8","30,4","34,5",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017
7,März,"32,4","33,8",----,"36,5","32,8","35,5",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017
8,April,"33,2","35,5","30,4","38,1","33,9","37,0",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017
9,Mai,"38,4","38,3",----,"39,2","38,4","37,7",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,August,"39,4","34,3",----,"38,7","38,5","38,0",2,15,Saarbruecken,1,Saarbruecken,2021
13,September,"38,6","34,1",----,"37,2","37,9","36,4",0,15,Saarbruecken,1,Saarbruecken,2021
14,Oktober,"34,9","37,0",----,"37,1","35,5","35,2",0,15,Saarbruecken,1,Saarbruecken,2021
15,November,"35,3","32,0",----,"34,8","34,7","33,9",0,15,Saarbruecken,1,Saarbruecken,2021


In [117]:
# backup downloaded data
df_city_year.to_csv('noise_2017-2021_month_clean.csv', index=False)

4,LTag(06‑18),LTagrand(18‑22),LNacht(00‑06+22-24),Lden(00-24),LTag(06‑22),Ldn(00-24),NAT (68 dBA)(Nacht),r_number,region,s_number,station,Date
5,----,"35,2","30,5","37,7","32,0","36,8",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-01-01
6,----,"34,3",----,"35,8","30,4","34,5",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-02-01
7,"32,4","33,8",----,"36,5","32,8","35,5",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-03-01
8,"33,2","35,5","30,4","38,1","33,9","37,0",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-04-01
9,"38,4","38,3",----,"39,2","38,4","37,7",0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...
12,"39,4","34,3",----,"38,7","38,5","38,0",2,15,Saarbruecken,1,Saarbruecken,2021-08-01
13,"38,6","34,1",----,"37,2","37,9","36,4",0,15,Saarbruecken,1,Saarbruecken,2021-09-01
14,"34,9","37,0",----,"37,1","35,5","35,2",0,15,Saarbruecken,1,Saarbruecken,2021-10-01
15,"35,3","32,0",----,"34,8","34,7","33,9",0,15,Saarbruecken,1,Saarbruecken,2021-11-01


In [ ]:
# Adding Year and month in one column

df_city_year['Date_temp']=df_city_year['Monat']
df_city_year['Date_temp'] = df_city_year['Date_temp'].replace(['Januar','Februar','März','April','Mai','Juni','Juli','August','September','Oktober','November','Dezember'],['01-01','02-01','03-01','04-01','05-01','06-01','07-01','08-01','09-01','10-01','11-01','12-01'])
df_city_year['Date'] = df_city_year['year']+"-"+df_city_year['Date_temp']
df_city_year.drop('Monat', inplace=True, axis=1)
df_city_year.drop('Date_temp', inplace=True, axis=1)
df_city_year.drop('year', inplace=True, axis=1)


df_city_year.info()

### Cleaning Noise per month

In [ ]:
import datetime
df_city_year.rename(columns={'LTag(06‑18)':'LTag(18)', 'LTagrand(18‑22)':'LTagrand','LNacht(00‑06+22-24)':'LNacht','Lden(00-24)':'Lden','LTag(06‑22)':'LTag(22)','Ldn(00-24)':'Ldn','NAT (68 dBA)(Nacht)':'NAT'},inplace=True)
df_city_year['Date'] = pd.to_datetime(df_city_year['Date'],format='%Y-%m-%d')

# transform Income column to a numerical
df_city_year['LTag(18)'] = df_city_year['LTag(18)'].str.replace('----', '')
df_city_year['LTag(18)'] = df_city_year['LTag(18)'].str.replace(',', '.')
df_city_year['LTag(18)'] = pd.to_numeric(df_city_year['LTag(18)'])

df_city_year['LTagrand'] = df_city_year['LTagrand'].str.replace('----', '')
df_city_year['LTagrand'] = df_city_year['LTagrand'].str.replace(',', '.')
df_city_year['LTagrand'] = pd.to_numeric(df_city_year['LTagrand'])

df_city_year['LNacht'] = df_city_year['LNacht'].str.replace('----', '')
df_city_year['LNacht'] = df_city_year['LNacht'].str.replace(',', '.')
df_city_year['LNacht'] = pd.to_numeric(df_city_year['LNacht'])

df_city_year['Lden'] = df_city_year['Lden'].str.replace('----', '')
df_city_year['Lden'] = df_city_year['Lden'].str.replace(',', '.')
df_city_year['Lden'] = pd.to_numeric(df_city_year['Lden'])

df_city_year['LTag(22)'] = df_city_year['LTag(22)'].str.replace('----', '')
df_city_year['LTag(22)'] = df_city_year['LTag(22)'].str.replace(',', '.')
df_city_year['LTag(22)'] = pd.to_numeric(df_city_year['LTag(22)'])

df_city_year['Ldn'] = df_city_year['Ldn'].str.replace('----', '')
df_city_year['Ldn'] = df_city_year['Ldn'].str.replace(',', '.')
df_city_year['Ldn'] = pd.to_numeric(df_city_year['Ldn'])

df_city_year['NAT'] = df_city_year['NAT'].str.replace('----', '')
df_city_year['NAT'] = df_city_year['NAT'].str.replace(',', '.')
df_city_year['NAT'] = pd.to_numeric(df_city_year['NAT'])

df_city_year.info()
df_city_year.head()

In [127]:
df_city_year.to_csv('noise_2017-2021_month_clean.csv', index=False)

In [3]:
# Import all necessary libraries
import psycopg2
from sql import engine #adjust this as necessary to match your sql.py connection methods
# Import connection parameters: host, port, database, user, password
import sqlalchemy
from sql import host, port, database, user, password

conn = psycopg2.connect(host=host,
                        port=port,
                        database=database,
                        user=user,
                        password=password)

print(conn)

<connection object at 0x10527f6d0; dsn: 'user=alexanderklar password=xxx dbname=postgres host=data-analytics-course.c8g8r1deus2v.eu-central-1.rds.amazonaws.com port=5432', closed: 0>


In [105]:
### Upload noise data to SQL

# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'noise_data_de'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_city.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        schema='cap_air',
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The noise_data_de table was imported successfully.


In [131]:
### Upload noise data_year to SQL

# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'noise_data_month_de'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_city_year.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        schema='cap_air',
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The noise_data_month_de table was imported successfully.


Load month again for selecting only certain stations of FRA, HH, HAN. 
- city itself and everything around under 15km
- 84, 100,296,297,298,80,81,68,82,291,292,83,300,301,92,188,190,99,212,1,10,7,11,5,6,15,8,7,10,14,12

In [5]:
#importing packages
import pandas as pd
import requests

In [6]:
# calling the noise per month table from sql

sql_query = 'SELECT * FROM cap_air.noise_data_month_de'
sql_query2 = 'SELECT * FROM cap_air.noise_data_de'
df_noise_month = pd.read_sql_query(sql_query, conn)
display(df_noise_month)
df_noise_day= pd.read_sql_query(sql_query2, conn)
display(df_noise_day)

,LTag(18),LTagrand,LNacht,Lden,LTag(22),Ldn,NAT,r_number,region,s_number,station,Date,date_tag
0,NaN,35.2,30.5,37.7,32.0,36.8,0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-01-01,2017-01-01
1,NaN,34.3,NaN,35.8,30.4,34.5,0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-02-01,2017-02-01
2,32.4,33.8,NaN,36.5,32.8,35.5,0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-03-01,2017-03-01
3,33.2,35.5,30.4,38.1,33.9,37.0,0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-04-01,2017-04-01
4,38.4,38.3,NaN,39.2,38.4,37.7,0,2,Berlin,62,Berlin/Friedrichshagen MSD xxx,2017-05-01,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,39.6,39.4,32.2,41.7,39.6,40.7,1,18,Munich,10,Markt Schwaben (ZMS2),2021-08-01,2021-08-01
13916,38.4,37.7,NaN,39.2,38.2,38.0,1,18,Munich,10,Markt Schwaben (ZMS2),2021-09-01,2021-09-01
13917,41.4,38.4,NaN,41.0,40.8,40.0,0,18,Munich,10,Markt Schwaben (ZMS2),2021-10-01,2021-10-01
13918,34.1,36.2,NaN,38.2,34.8,37.1,0,18,Munich,10,Markt Schwaben (ZMS2),2021-11-01,2021-11-01


,Date,LTag(18),LTagrand,LNacht,Lden,LTag(22),Ldn,NAT,r_number,region,s_number,station
0,2018-05-15,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
1,2018-05-16,35.2,NaN,NaN,32.2,34.0,32.2,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
2,2018-05-17,46.8,47.1,39.4,49.1,46.9,47.9,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
3,2018-05-18,40.3,NaN,NaN,39.1,39.0,39.1,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
4,2018-05-19,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
...,...,...,...,...,...,...,...,...,...,...,...,...
405784,2018-05-10,44.7,44.8,34.7,46.1,44.7,44.7,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
405785,2018-05-11,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
405786,2018-05-12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx
405787,2018-05-13,46.9,43.0,NaN,45.7,46.2,44.7,0.0,1,Frankfurt,248,Maintal/Doernigheim xxx


In [14]:
# select the relevant stations only

region_ids = [1,16,5]
station_ids = [84, 100,296,297,298,80,81,68,82,291,292,83,300,301,92,188,190,99,212,1,10,7,11,5,6,15,8,7,10,14,12]

df_noise_month_stations = df_noise_month[df_noise_month['r_number'].isin(region_ids) & df_noise_month['s_number'].isin(station_ids)]
display(df_noise_month_stations.sort_values('s_number'))

df_noise_day_stations = df_noise_day[df_noise_day['r_number'].isin(region_ids) & df_noise_day['s_number'].isin(station_ids)]
display(df_noise_day_stations.sort_values('s_number'))

,LTag(18),LTagrand,LNacht,Lden,LTag(22),Ldn,NAT,r_number,region,s_number,station,Date,date_tag
7839,NaN,32.6,NaN,32.8,NaN,30.9,0,1,Frankfurt,1,Eppstein/Vockenhausen xxx,2020-06-01,2020-06-01
1979,49.0,48.6,48.1,54.7,48.9,54.3,326,5,Hannover,1,Garbsen 1/Schlo&szlig;-Ricklingen,2017-08-01,2017-08-01
1980,48.5,48.4,47.4,54.1,48.5,53.6,270,5,Hannover,1,Garbsen 1/Schlo&szlig;-Ricklingen,2017-09-01,2017-09-01
1981,48.5,48.7,47.8,54.4,48.6,54.0,312,5,Hannover,1,Garbsen 1/Schlo&szlig;-Ricklingen,2017-10-01,2017-10-01
1982,47.0,46.6,43.8,51.1,46.9,50.5,90,5,Hannover,1,Garbsen 1/Schlo&szlig;-Ricklingen,2017-11-01,2017-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10622,61.4,60.7,52.8,62.9,61.2,61.8,95,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx,2021-09-01,2021-09-01
10623,62.1,61.3,51.6,63.0,61.9,61.8,139,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx,2021-10-01,2021-10-01
10624,63.3,61.5,53.4,64.0,62.9,63.0,219,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx,2021-11-01,2021-11-01
8062,NaN,NaN,NaN,NaN,NaN,NaN,----,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx,2020-01-01,2020-01-01


,Date,LTag(18),LTagrand,LNacht,Lden,LTag(22),Ldn,NAT,r_number,region,s_number,station
231431,2021-08-31,NaN,32.1,NaN,30.6,NaN,NaN,0.0,1,Frankfurt,1,Eppstein/Vockenhausen xxx
38814,2018-12-20,36.6,38.1,31.4,40.1,37.0,39.0,0.0,1,Frankfurt,1,Eppstein/Vockenhausen xxx
38813,2018-12-19,35.0,35.2,NaN,36.3,35.0,34.9,0.0,1,Frankfurt,1,Eppstein/Vockenhausen xxx
38812,2018-12-18,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,Frankfurt,1,Eppstein/Vockenhausen xxx
38811,2018-12-17,30.9,36.9,NaN,35.7,33.3,33.0,0.0,1,Frankfurt,1,Eppstein/Vockenhausen xxx
...,...,...,...,...,...,...,...,...,...,...,...,...
145129,2020-06-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx
145128,2020-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx
145127,2020-06-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx
145138,2020-06-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Frankfurt,301,Frankfurt/Sachsenhausen 3 (dBC) xxx


In [15]:
### Upload noise data_day_stations to SQL

# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'noise_day_stations'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_noise_day_stations.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        schema='cap_air',
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The noise_day_stations table was imported successfully.


In [16]:
### Upload noise data_day_stations to SQL

# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'noise_month_stations'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_noise_month_stations.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        schema='cap_air',
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The noise_month_stations table was imported successfully.
